In [3]:
# (bucket_name='sagemaker-tables-detector', key='tables-detector-data.pickle.zip')

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

from sagemaker.mxnet.model import MXNetModel,MXNetPredictor

sagemaker_model = MXNetModel(model_data=None,
                             role=role, entry_point="tables_detector-class-hocr-template-with_hp_hw-Copy1.py",py_version="py3")


predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='ml.t2.medium' ) # , instance_type='local'
#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-09-16-31-21-144')




UndefinedModelAttributeError: "serviceId" not defined in the metadata of the model: ServiceModel(sagemaker-runtime)

# Inference from previously deployed endpoint

In [64]:
import sagemaker
from sagemaker import get_execution_role
import json
import pandas as pd
sagemaker_session = sagemaker.Session()

role = get_execution_role()

from sagemaker.mxnet.model import MXNetModel, MXNetPredictor

#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-19-19-28-10-154') # Writing files as is. If s3 files are in subfolders, the name includes the subfolder which doesn't exist
#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-23-14-22-26-095') # Feeding streams to cv2.imread. Not working
#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-23-14-50-27-786') # Split name to get file name only
#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-24-13-03-50-315') # Harathi solution with bytes_data
# predictor =MXNetPredictor('sagemaker-mxnet-2018-10-23-15-50-43-104') # Fixed xml and image file names, to avoid special char issue

#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-25-19-59-43-168') # Returning coordinates
#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-29-14-04-10-452') # Filter on known headers: generic mnot per col
#predictor =MXNetPredictor('sagemaker-mxnet-2018-10-31-12-56-07-634') # Filter on known headers: specific per col
predictor =MXNetPredictor("sagemaker-mxnet-2018-11-12-13-38-23-326") # HP+HW+word level info from tesseract HOCR

image_file_name = "Accident Claim - 2_3.tiff"
hocr_file_name = "Accident Claim - 2-pdfinput-sandwich-3.hocr"

#image_file_name = "Accident-Handwritten_V3.tiff"
#xml_file_name = "Accident-Handwritten_V3.xml"

#  unum-files
data = {"bucket": 'sagemaker-tables-detector', "hocr_file": hocr_file_name,"image_file":image_file_name}
#data = {"bucket": 'sagemaker-tables-detector', "xml_file": xml_file_name,"image_file":image_file_name}

response = predictor.predict(data)
#print(response)
print("there are ",len(response['data']),"tables in",hocr_file_name.split(".")[0])
print(json.dumps(response, sort_keys=True, indent=4).encode('utf-8').decode('utf-8'))


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (504) from model with message "<html>
<head><title>504 Gateway Time-out</title></head>
<body bgcolor="white">
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.10.3 (Ubuntu)</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-mxnet-2018-11-12-13-38-23-326 in account 620580205565 for more information.

In [81]:
print(len(response['coordinates']))

6


In [82]:
response['coordinates']

[{'y1': 800, 'y2': 1100, 'x2': 4926, 'x1': 1349},
 {'y1': 2150, 'y2': 3650, 'x2': 5526, 'x1': 749},
 {'y1': 3900, 'y2': 4200, 'x2': 5526, 'x1': 749},
 {'y1': 4500, 'y2': 5150, 'x2': 5526, 'x1': 749},
 {'y1': 5550, 'y2': 5750, 'x2': 5526, 'x1': 749},
 {'y1': 6750, 'y2': 6900, 'x2': 5076, 'x1': 1199}]

In [43]:
import sagemaker
from sagemaker import get_execution_role
import json
import pandas as pd
sagemaker_session = sagemaker.Session()

role = get_execution_role()

from sagemaker.mxnet.model import MXNetModel,MXNetPredictor

predictor =MXNetPredictor('sagemaker-mxnet-2018-10-19-19-28-10-154')
xml_file_name = "Accident-Handwritten_V3.xml"
image_file_name = "Accident-Handwritten_V3.tiff"

data = {"bucket": 'sagemaker-tables-detector', "xml_file": xml_file_name,"image_file":image_file_name}

response = predictor.predict(data)
#print(response)
print("there are ",len(response['data']),"tables in",xml_file_name.split(".")[0])
print(json.dumps(response, sort_keys=True, indent=4).encode('utf-8').decode('utf-8'))


there are  5 tables in Accident-Handwritten_V3
{
    "data": [
        {
            "col1": {
                "0": "Cail toll-free Monday through Friday, 8 a.m. to 8 p.m. Eastern Time.",
                "1": "",
                "2": "ATTENDING PHYSICIAN STATEMENT"
            },
            "col2": {
                "0": "",
                "1": "",
                "2": ""
            }
        },
        {
            "col1": {
                "0": "ACCIDENTDETAILS",
                "1": "",
                "2": "Is this condition the result of an accidental injury? \u2014 Yes O1 No If yes, date of accident (mm/dd/yy) on"
            },
            "col2": {
                "0": "",
                "1": "",
                "2": ""
            }
        },
        {
            "col1": {
                "0": "Dates of Diagnosis",
                "1": "Service (including Code (ICD)",
                "2": "Confinement)",
                "3": "",
                "4": ""
            },
  

In [25]:
import boto3

bucket = 'sagemaker-tables-detector'

image_file_name = "Accident-Handwritten_V3-0.jpg"
hocr_file_name = "Accident-Handwritten_V3-sandwich.hocr"

#image_file_name = "Accident-Handwritten_V3.tiff"
#xml_file_name = "Accident-Handwritten_V3.xml"

#  unum-files
data = {"bucket": 'sagemaker-tables-detector', "hocr_file": hocr_file_name,"image_file":image_file_name}
# xml_file_name = "Accident-Handwritten_V3.xml"
# image_file_name = "Accident-Handwritten_V3.tiff"

# getting data
s3 = boto3.resource('s3', aws_access_key_id='', aws_secret_access_key='')  # new key
xml_file_stream = s3.Object(bucket_name=bucket, key=hocr_file_name)
image_file_stream = s3.Object(bucket_name=bucket, key=image_file_name)

# write from binary stream
# xml_file_name = xml_file_name.split('/')[-1]
hocr_file_name = 'hocr_file.html'
with open(hocr_file_name, 'wb') as f:
    f.write(xml_file_stream.get()["Body"].read())

# write from binary stream
# image_file_name = image_file_name.split('/')[-1]
image_file_name = 'image_file_name.tif'
with open(image_file_name, 'wb') as f:
    f.write(image_file_stream.get()["Body"].read())

In [27]:
import time

import boto3
import sagemaker
from sagemaker import RealTimePredictor
from sagemaker.mxnet import MXNetPredictor
from sagemaker.predictor import json_serializer, json_deserializer


def upload_file(s3, bucket, local_path_to_file, s3_path_to_file):
    s3.Bucket(bucket).upload_file(local_path_to_file, s3_path_to_file)


def download_file(s3, bucket, local_path_to_file, s3_path_to_file):
    file_binary_stream = s3.Object(bucket_name=bucket, key=s3_path_to_file)
    with open(local_path_to_file, 'wb') as f:
        f.write(file_binary_stream.get()["Body"].read())


session = boto3.Session(region_name='us-west-2', aws_access_key_id='', aws_secret_access_key='')
sagemaker_session = sagemaker.Session(boto_session=session)

loc_predictor = MXNetPredictor('sagemaker-mxnet-2018-11-05-22-58-03-333', sagemaker_session)


# Initialize handwriting model
class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)


hw_predictor = JSONPredictor('pytorch-handwriting-ocr-2018-10-31-19-34-48-492', sagemaker_session)

hp_predictor = MXNetPredictor('sagemaker-mxnet-2018-11-03-23-32-01-918', sagemaker_session)

# Initialize spell corrector model
spell_predictor = JSONPredictor('spell-corrector-2018-10-06-21-05-31-553', sagemaker_session)

# Upload images to S3

# print("Uploading images...")
# #Provide your access key and secret access key
s3 = boto3.resource('s3', aws_access_key_id='', aws_secret_access_key='')

download_file(s3,'sagemaker-tables-detector',"Accident-Handwritten_V3-sandwich.hocr","Accident-Handwritten_V3-sandwich.hocr")

In [29]:
download_file(s3,'sagemaker-tables-detector',"Accident-Handwritten_V3-sandwich.hocr","Accident-Handwritten_V3-sandwich.hocr")